# DS-SF-36 | 10 | Linear Regression, Part 3 | Assignment | Starter Code

In [1]:
import os

import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 10)

from sklearn import feature_selection, linear_model

In [2]:
df = pd.read_csv(os.path.join('..', 'datasets', 'dataset-10-credit.csv'))

In [3]:
df

,Income,Rating,Cards,Age,Education,Gender,Student,Married,Ethnicity,Balance
0,14.891,283,2,34,11,Male,No,Yes,Caucasian,333
1,106.025,483,3,82,15,Female,Yes,Yes,Asian,903
2,104.593,514,4,71,11,Male,No,No,Asian,580
3,148.924,681,3,36,11,Female,No,No,Asian,964
4,55.882,357,2,68,16,Male,No,Yes,Caucasian,331
...,...,...,...,...,...,...,...,...,...,...
395,12.096,307,3,32,13,Male,No,Yes,Caucasian,560
396,13.364,296,5,65,17,Male,No,No,African American,480
397,57.872,321,5,67,12,Female,No,Yes,Caucasian,138
398,37.728,192,1,44,13,Male,No,Yes,Caucasian,0


In [4]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Income,400.0,45.218885,35.244273,10.354,21.00725,33.1155,57.47075,186.634
Rating,400.0,354.940000,154.724143,93.000,247.25000,344.0000,437.25000,982.000
Cards,400.0,2.957500,1.371275,1.000,2.00000,3.0000,4.00000,9.000
Age,400.0,55.667500,17.249807,23.000,41.75000,56.0000,70.00000,98.000
Education,400.0,13.450000,3.125207,5.000,11.00000,14.0000,16.00000,20.000
Balance,400.0,520.015000,459.758877,0.000,68.75000,459.5000,863.00000,1999.000


A description of the dataset is as follows:

- Income (in thousands of dollars)
- Rating: Credit score rating
- Cards: Number of credit cards owned
- Age
- Education: Years of education
- Gender: Male/Female
- Student: Yes/No
- Married: Yes/No
- Ethnicity: African American/Asian/Caucasian
- Balance: Average credit card debt

> ## Question 1.  Let's explore the quantitative variables that affect `Balance`.  From your preliminary analysis, which 2 variables seem to affect `Balance` the most?  Our goal is interpretation; can we use these 2 variables simultaneously?  Why or why not?

In [5]:
# TODO
df.corr()

,Income,Rating,Cards,Age,Education,Balance
Income,1.000000,0.791378,-0.018273,0.175338,-0.027692,0.463656
Rating,0.791378,1.000000,0.053239,0.103165,-0.030136,0.863625
Cards,-0.018273,0.053239,1.000000,0.042948,-0.051084,0.086456
Age,0.175338,0.103165,0.042948,1.000000,0.003619,0.001835
Education,-0.027692,-0.030136,-0.051084,0.003619,1.000000,-0.008062
Balance,0.463656,0.863625,0.086456,0.001835,-0.008062,1.000000


Answer: Rating and income are highly correlated to Balance, at 46% and 86%, respectively.  Rating and income are also very highly correlated at 79%.  There is a good chance the these two variables will cause colinearity.  

In [6]:
df.isnull().sum()

Income       0
Rating       0
Cards        0
Age          0
Education    0
Gender       0
Student      0
Married      0
Ethnicity    0
Balance      0
dtype: int64

> ## Question 2.  `Ethnicity`, `Gender`, `Married`, and `Student` are categorical variables.  Go ahead and create dummy variables for all of them.

In [7]:
# TODO
Ethnicity = pd.get_dummies(df.Ethnicity,prefix = 'Ethnicity')
Ethnicity.head()

,Ethnicity_African American,Ethnicity_Asian,Ethnicity_Caucasian
0,0,0,1
1,0,1,0
2,0,1,0
3,0,1,0
4,0,0,1


In [8]:
Gender = pd.get_dummies(df.Gender,prefix = 'Gender')
Gender.head()

,Gender_Female,Gender_Male
0,0,1
1,1,0
2,0,1
3,1,0
4,0,1


In [9]:
Married = pd.get_dummies(df.Married,prefix = 'Married')
Married.head()

,Married_No,Married_Yes
0,0,1
1,0,1
2,1,0
3,1,0
4,0,1


In [10]:
Student = pd.get_dummies(df.Student,prefix = 'Student')
Student.head()

,Student_No,Student_Yes
0,1,0
1,0,1
2,1,0
3,1,0
4,1,0


In [11]:
df = pd.merge(df, Ethnicity,left_index=True, right_index=True)
df = pd.merge(df, Gender, left_index=True, right_index=True )
df = pd.merge(df, Married, left_index=True, right_index=True )
df = pd.merge(df, Student, left_index=True, right_index=True )
df.head()

,Income,Rating,Cards,Age,Education,...,Gender_Male,Married_No,Married_Yes,Student_No,Student_Yes
0,14.891,283,2,34,11,...,1,0,1,1,0
1,106.025,483,3,82,15,...,0,0,1,0,1
2,104.593,514,4,71,11,...,1,1,0,1,0
3,148.924,681,3,36,11,...,0,1,0,1,0
4,55.882,357,2,68,16,...,1,0,1,1,0


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 19 columns):
Income                        400 non-null float64
Rating                        400 non-null int64
Cards                         400 non-null int64
Age                           400 non-null int64
Education                     400 non-null int64
Gender                        400 non-null object
Student                       400 non-null object
Married                       400 non-null object
Ethnicity                     400 non-null object
Balance                       400 non-null int64
Ethnicity_African American    400 non-null uint8
Ethnicity_Asian               400 non-null uint8
Ethnicity_Caucasian           400 non-null uint8
Gender_Female                 400 non-null uint8
Gender_Male                   400 non-null uint8
Married_No                    400 non-null uint8
Married_Yes                   400 non-null uint8
Student_No                    400 non-null uint8
Student

> ## Question 3.  Using _sklearn_'s linear regression, predict `Balance` using `Income`, `Cards`, `Age`, `Education`, `Gender`, and `Ethnicity`

First, find the coefficients of your regression line.

In [14]:
# TODO
X = df[ ['Income','Cards','Age','Education','Gender_Male','Gender_Female', \
         'Ethnicity_Asian','Ethnicity_Caucasian','Ethnicity_African American'] ]
y = df.Balance

model = linear_model.LinearRegression().fit(X, y)

Then, find the p-values of your F-values' models.  You have a few variables, so try to show your p-values alongside the names of the variables.  (https://docs.python.org/2/library/functions.html#zip)

In [15]:
# TODO
def summary(X, y, model):
    _, f_pvalues = feature_selection.f_regression(X, y)

    print 'R^2 =', model.score(X, y)
    print

    print 'Coefficients'
    print '- beta_0 (Intercept) = {}'.format(model.intercept_)

    for i, coef in enumerate(model.coef_):
        print '- beta_{} ({}) = {} (p-value = {})'.format(i + 1, X.columns[i], coef, f_pvalues[i])

        
summary(X, y, model)

R^2 = 0.232312608335

Coefficients
- beta_0 (Intercept) = 242.581385219
- beta_1 (Income) = 6.27995894353 (p-value = 1.03088580259e-22)
- beta_2 (Cards) = 33.6295350792 (p-value = 0.0841765555994)
- beta_3 (Age) = -2.32970547308 (p-value = 0.97081387233)
- beta_4 (Education) = 1.64553607303 (p-value = 0.872306401567)
- beta_5 (Gender_Male) = -13.5627156158 (p-value = 0.668516105503)
- beta_6 (Gender_Female) = 13.5627156158 (p-value = 0.668516105503)
- beta_7 (Ethnicity_Asian) = -5.52234599168 (p-value = 0.844895644362)
- beta_8 (Ethnicity_Caucasian) = 4.49866120235 (p-value = 0.947727511397)
- beta_9 (Ethnicity_African American) = 1.02368478932 (p-value = 0.784430319088)


> ## Question 4.  Which of your coefficients are significant at the 5% significance level?

Answer: Income is significant at 5% significance level.

> ## Question 5.  What is your model's $R^2$?

In [16]:
# TODO
model.score(X,y)

0.23231260833540443

> ## Question 6.  How do we interpret this value?

Answer: 23% of the total variance in Balance can be explained by the model.

> ## Question 7.  Now let's focus on the two most significant variables from your previous model and re-run your regression model.

In [17]:
# TODO
X = df[ ['Income','Cards'] ]
y = df.Balance

model = linear_model.LinearRegression().fit(X, y)
summary(X,y, model)


R^2 = 0.223991751622

Coefficients
- beta_0 (Intercept) = 151.329946349
- beta_1 (Income) = 6.07099859467 (p-value = 1.03088580259e-22)
- beta_2 (Cards) = 31.8381289478 (p-value = 0.0841765555994)


> ## Question 8.  In comparison to the previous model, did the $R^2$ increase or decrease?  Why?

In [18]:
# TODO
model.score(X,y)

0.22399175162249518

Answer: R square decreased slightly from .23 to .22.  The more variables you have in the model, the higher R square will be.  When we decreased the model from previous version to just 2 variables, the R square decreases.

> ## Question 9.  Now let's regress `Balance` on `Gender` alone.  After running your linear regressions, do you have enough evidence to claim that females have more balance than males?  (Hint: Look at the p-value of the Gender coefficient.  If it is significant then you will have evidence to support that claim, otherwise you cannot support the statement.)

In [19]:
# TODO
X = df[ ['Gender_Male', 'Gender_Female'] ]
y = df.Balance

model = linear_model.LinearRegression().fit(X, y)
summary(X,y,model)


R^2 = 0.000461132964496

Coefficients
- beta_0 (Intercept) = 519.669670346
- beta_1 (Gender_Male) = -9.86656153788 (p-value = 0.668516105503)
- beta_2 (Gender_Female) = 9.86656153788 (p-value = 0.668516105503)


Answer: (If your answer is yes, interpret the results).  We don't have enough evidence to claim that females have more balance than males.

> ## Question 10.  Now let's regress `Balance` on `Ethnicity`.  After running your linear regressions, do you have enough evidence to claim that some ethnic groups carry more balance than others?

In [20]:
# TODO
X = df[ ['Ethnicity_Asian','Ethnicity_Caucasian','Ethnicity_African American'] ]
y = df.Balance

model = linear_model.LinearRegression().fit(X, y)
summary(X,y,model)

R^2 = 0.000218807443049

Coefficients
- beta_0 (Intercept) = 520.603737642
- beta_1 (Ethnicity_Asian) = -8.29001215226 (p-value = 0.844895644362)
- beta_2 (Ethnicity_Caucasian) = -2.10625020527 (p-value = 0.947727511397)
- beta_3 (Ethnicity_African American) = 10.3962623575 (p-value = 0.784430319088)


Answer: (If your answer is yes, interpret the results).  There is no evidence to claim that some ethnic groups carry more balance than others.

> ## Question 11.  Finally let's regress `Balance` on `Student`.  After running your linear regressions, do you have enough evidence to claim that students carry more balance than non-students?

In [21]:
# TODO
X = df[ ['Student_Yes','Student_No'] ]
y = df.Balance

model = linear_model.LinearRegression().fit(X, y)
summary(X,y,model)

R^2 = 0.0670900898871

Coefficients
- beta_0 (Intercept) = 678.597222222
- beta_1 (Student_Yes) = 198.227777778 (p-value = 1.48773410773e-07)
- beta_2 (Student_No) = -198.227777778 (p-value = 1.48773410773e-07)


Answer: (If your answer is yes, interpret the results).  Yes, student carries on average $396 more in balance than non-students.

> ## Question 12.  No let's consider the effect of `Student` and `Income` on `Balance` simultaneously.  Are all the coefficients significant?

In [22]:
# TODO

Answer: (If your answer is yes, interpret the results).  

> ## Question 13.  No let's consider the interaction effect of `Student` and `Income` on `Balance` simultaneously.  Are all the coefficients significant?  It they are, write down your regression model below

(First generate a new variable for the interaction term)

In [23]:
# TODO

Answer: 

> ## Question 14.  Is there any income level at which students and non-students on average carry same level of balance?

Answer: 